In [43]:
# Get all Team IDs
from nba_api.stats.static import teams
import pandas as pd
pd.set_option('display.max_columns', None) 

team_dict = teams.get_teams()
id_name_pairs = [(team['id'], team['full_name']) for team in team_dict]
ids = [pair[0] for pair in id_name_pairs]
#ids

In [44]:
# Get all the Warriors Stats
from nba_api.stats.endpoints import leaguegamefinder

all_teams_data_frames = []

# Loop through each team ID and fetch the corresponding team stats
for team_id in ids:
    team_games = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id)
    team_data_frame = team_games.get_data_frames()[0]
    all_teams_data_frames.append(team_data_frame)

In [50]:
all_teams_data_frames[0]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22023,1610612737,ATL,Atlanta Hawks,0022300521,2024-01-10,ATL vs. PHI,W,265,139,48,102,0.471,12,34,0.353,31,42,0.738,16,39,55,32,6.0,8,12,21,7.0
1,22023,1610612737,ATL,Atlanta Hawks,0022300499,2024-01-07,ATL @ ORL,L,265,110,42,91,0.462,9,30,0.300,17,29,0.586,8,42,50,21,9.0,10,12,23,-7.0
2,22023,1610612737,ATL,Atlanta Hawks,0022300480,2024-01-05,ATL @ IND,L,238,116,40,93,0.430,10,42,0.238,26,30,0.867,14,25,39,21,6.0,3,14,11,-34.0
3,22023,1610612737,ATL,Atlanta Hawks,0022300467,2024-01-03,ATL vs. OKC,W,240,141,48,95,0.505,14,39,0.359,31,33,0.939,15,34,49,30,8.0,5,11,21,3.0
4,22023,1610612737,ATL,Atlanta Hawks,0022300445,2023-12-31,ATL @ WAS,W,240,130,47,95,0.495,12,32,0.375,24,37,0.649,12,46,58,27,5.0,3,12,18,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3644,21983,1610612737,ATL,Atlanta Hawks,0028300058,1983-11-06,ATL @ MIL,L,240,84,32,78,0.410,0,2,0.000,20,30,0.667,11,26,37,18,4.0,5,14,24,NaN
3645,21983,1610612737,ATL,Atlanta Hawks,0028300041,1983-11-04,ATL vs. CHI,W,240,103,42,86,0.488,1,1,1.000,18,26,0.692,19,27,46,31,14.0,13,18,27,NaN
3646,21983,1610612737,ATL,Atlanta Hawks,0028300027,1983-11-01,ATL vs. WAS,W,240,95,38,81,0.469,0,0,NaN,19,30,0.633,12,29,41,20,7.0,10,16,34,NaN
3647,21983,1610612737,ATL,Atlanta Hawks,0028300014,1983-10-29,ATL vs. DET,W,240,117,49,94,0.521,0,1,0.000,19,30,0.633,27,21,48,28,14.0,7,23,35,NaN
